#### <b> Để tránh việc `hard-code` ta sẽ định nghĩa một số biến `constanst` sau:</b>

In [7]:
# Tên file chứa data
DATA_FILE_NAME = "Sample - Superstore.csv"
# Tên các column
ROW_ID = 'ROW ID'
ORDER_ID = 'Order ID'
ORDER_DATE = 'Order Date'
SHIP_DATE = 'Ship Date'
SHIP_MODE = 'Ship Mode'
CUSTOMER_ID = 'Customer ID'
CUSTOMER_NAME = 'Customer Name'
SEGMENT = 'Segment'
COUNTRY = 'Country'
CITY = 'City'
STATE = 'State'
POSTAL_CODE = 'Postal Code'
REGION = 'REGION'
PRODUCT_ID = 'Product ID'
CATEGORY = 'Category'
SUB_CATEGORY = 'Sub-Category'
PRODUCT_NAME = 'Product Name'
SALES = 'Sales'
QUANTITY = 'Quantity'
DISCOUNT = 'Discount'
PROFIT = 'Profit'

#### <b>1 <span style='color:#B21010'>|</span> Đọc dữ liệu và tính số dòng và cột</b>

👉 Đầu tiên ta import, cài đặt các thư viện cần thiết.

In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

👉 Tiếp đến đọc file `"Sample - Superstore.csv"` vào dataframe `df` và in ra 5 dòng đầu tiên của dataframe.

In [9]:

df = pd.read_csv(DATA_FILE_NAME, encoding= 'unicode_escape')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


👉 Tính số dòng và số cột và lưu vào 2 biến `num_rows` và `num_cols`.

In [10]:
num_rows, num_cols = df.shape
print(f'Number of rows: {num_rows}\nNumber of columns: {num_cols}')

Number of rows: 9994
Number of columns: 21


#### <b>2 <span style='color:#B21010'>|</span> Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?</b>

* Bộ dữ liệu chứa thông tin liên quan đến Bán hàng, lợi nhuận và các nhân tố thú vị khác của một siêu thị lớn.
* Mỗi dòng là thông tin của một sản phẩm được mua bao gồm các thông tin về sản phẩm và thông tin về khách hàng mua sản phẩm.
<br>
**=> Vì thế sẽ không xuất hiện dòng có ý nghĩa khác.** 

#### <b>3 <span style='color:#B21010'>|</span> Dữ liệu có các dòng bị lặp không?</b>

👉 Ta kiểm tra xem có dòng nào bị lặp không bằng cách sử dụng các phương thức `duplicated()` và `any()` trên dataframe `df` và lưu kết quả vào biến `have_duplicated_rows`. Biến này sẽ có giá trị True nếu dữ liệu có các dòng bị lặp và có giá trị False nếu ngược.

In [11]:
have_duplicate_rows = df.duplicated().any()
have_duplicate_rows

False

💡 Ta thấy rằng dữ liệu không có dòng nào bị trùng lặp.

#### <b>4 <span style='color:#B21010'>|</span> Tỷ lệ giá trị thiếu và thống kê mô tả của từng cột</b>

Để tính tỷ lệ giá trị thiếu, sử dụng phương thức `isna()` và `mean()` và lưu vào biến `missing_rate`

Để thống kê mô tả mỗi cột, phương thức `describe()`.
Trong đó:
* `count`: số lượng giá trị không bị thiếu trong cột.
* `mean` : giá trị trung bình của các giá trị trong cột.
* `std` : độ lệch chuẩn của các giá trị trong cột.
* `min` : giá trị nhỏ nhất trong cột.
* `25%`, `50%`, `75%` : các **`phân vị`** tương ứng với các mức phân chia dữ liệu là `25%`, `50%` và `75%`.
* `max` : giá trị lớn nhất trong cột.

👉 Thống kê các cột `numerical`

In [12]:
describe = df.describe()
missing_rates = df[describe.columns].isna().mean()
missing_rates.name = 'missing_rate'
describe = pd.concat([describe, missing_rates.to_frame().T])
describe

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000
missing_rate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


👉 Thống kê các cột `category`

In [13]:
describe = df.describe(include=['O'])
missing_rates = df[describe.columns].isna().mean()
missing_rates.name = 'missing_rate'
describe = pd.concat([describe, missing_rates.to_frame().T])
describe

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name
count,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994
unique,5009,1237,1334,4,793,793,3,1,531,49,4,1862,3,17,1850
top,CA-2017-100111,9/5/2016,12/16/2015,Standard Class,WB-21850,William Brown,Consumer,United States,New York City,California,West,OFF-PA-10001970,Office Supplies,Binders,Staple envelope
freq,14,38,35,5968,37,37,5191,9994,915,2001,3203,19,6026,1523,48
missing_rate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### <b>5 <span style='color:#B21010'>|</span> Kiểu dữ liệu của mỗi cột</b>

👉 Ta sử dụng phương thức `dtypes` trên dataframe `df` để xem kiểu dữ liệu của mỗi cột. Kết quả được lưu vào series `col_dtypes`; series này có index là tên các cột và giá trị là kiểu dữ liệu của các cột tương ứng.

In [21]:
col_dtype = df.dtypes
col_dtype

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
dtype: object

👉 Ta sử dụng phương thức `select_dtypes` để liệt kê các cột kiểu `numerical` và `category`

In [19]:
cat_coulmns  = df.select_dtypes(['object']).columns

num_coulmns  = df.select_dtypes(['number']).columns

print(cat_coulmns)
print(num_coulmns)

Index(['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Segment', 'Country', 'City', 'State', 'Region',
       'Product ID', 'Category', 'Sub-Category', 'Product Name'],
      dtype='object')
Index(['Row ID', 'Postal Code', 'Sales', 'Quantity', 'Discount', 'Profit'], dtype='object')


👉 Chuyển `Order Date` và `Ship Date` sang kiểu dữ liệu `datetime`

In [24]:
df['Order Date'] = df['Order Date'].astype('datetime64[ns]')
df['Ship Date'] = df['Ship Date'].astype('datetime64[ns]')